
# 🍽️ Zomato Restaurant Data Analysis & Feature Engineering

This notebook performs **end-to-end exploratory data analysis (EDA)** and **feature engineering**
on the Zomato restaurant dataset.

The goal is to transform raw, messy data into a **clean, model-ready dataset**
suitable for downstream machine learning tasks.



## 1. Imports & Setup


In [20]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler



## 2. Load Dataset

> Dataset is not included due to size and licensing constraints.
> Download `zomato.csv` from a public source (e.g. Kaggle) and place it in the project root.


In [27]:

df = pd.read_csv("zomato.csv")

print("Dataset shape:", df.shape)
df.head()


Dataset shape: (51717, 17)


,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari



## 3. Initial Data Inspection


In [22]:

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51717 entries, 0 to 51716
Data columns (total 17 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   url                          51717 non-null  object
 1   address                      51717 non-null  object
 2   name                         51717 non-null  object
 3   online_order                 51717 non-null  object
 4   book_table                   51717 non-null  object
 5   rate                         43942 non-null  object
 6   votes                        51717 non-null  int64 
 7   phone                        50509 non-null  object
 8   location                     51696 non-null  object
 9   rest_type                    51490 non-null  object
 10  dish_liked                   23639 non-null  object
 11  cuisines                     51672 non-null  object
 12  approx_cost(for two people)  51371 non-null  object
 13  reviews_list                 51

In [23]:

df.isnull().sum()


url                                0
address                            0
name                               0
online_order                       0
book_table                         0
rate                            7775
votes                              0
phone                           1208
location                          21
rest_type                        227
dish_liked                     28078
cuisines                          45
approx_cost(for two people)      346
reviews_list                       0
menu_item                          0
listed_in(type)                    0
listed_in(city)                    0
dtype: int64


## 4. Data Cleaning


In [30]:
df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(' ', '_')
)

In [35]:

# Remove duplicates
df.drop_duplicates(inplace=True)

# Clean ratings
df['rate'] = pd.to_numeric(
    df['rate']
    .replace(['NEW', '-', 'None', None], np.nan)
    .astype(str)
    .str.split('/')
    .str[0],
    errors='coerce'
)

# Handle missing values safely
df['rate'] = df['rate'].fillna(df['rate'].mean())

if not df['location'].mode().empty:
    df['location'] = df['location'].fillna(df['location'].mode()[0])
else:
    df['location'] = df['location'].fillna('Unknown')

df['cuisines'] = df['cuisines'].fillna('Unknown')


# Clean cost column
df['approx_cost(for_two_people)'] = (
    df['approx_cost(for_two_people)']
    .str.replace(',', '')
    .astype(float)
)



## 5. Feature Engineering


In [37]:

# Binary features
df['Has_Online'] = (df['online_order'] == 'Yes').astype(int)
df['Has_Table'] = (df['book_table'] == 'Yes').astype(int)

# Expense indicator
df['Is_Expensive'] = (
    df['approx_cost(for_two_people)'] >
    df['approx_cost(for_two_people)'].median()
).astype(int)

# Cuisine count
df['Cuisine_Count'] = df['cuisines'].str.split(',').str.len()

# Aggregated ratings
df['avg_rating_location'] = df.groupby('location')['rate'].transform('mean')
df['avg_rating_cuisines'] = df.groupby('cuisines')['rate'].transform('mean')



## 6. Outlier Handling (IQR Clipping)


In [38]:

def clip_outliers(series):
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    return series.clip(lower, upper)

df['rate'] = clip_outliers(df['rate'])
df['approx_cost(for_two_people)'] = clip_outliers(df['approx_cost(for_two_people)'])
df['votes'] = clip_outliers(df['votes'])



## 7. Handling High-Cardinality Categorical Features


In [39]:

def group_rare(series, top_n=10):
    top_categories = series.value_counts().index[:top_n]
    return series.apply(lambda x: x if x in top_categories else 'Other')

df['rest_type'] = group_rare(df['rest_type'], 10)
df['location'] = group_rare(df['location'], 10)



## 8. Dish Popularity Feature Engineering


In [43]:

all_dishes = df['dish_liked'].dropna().str.split(',').explode()
top_dishes = all_dishes.value_counts().index[:10]

def filter_top_dishes(dishes):
    if pd.isna(dishes):
        return 'Other'
    filtered = [d for d in dishes.split(',') if d in top_dishes]
    return ','.join(filtered) if filtered else 'Other'

df['dish_liked_grouped'] = df['dish_liked'].apply(filter_top_dishes)
df['cuisine_count'] = df['cuisines'].str.split(',').apply(len)



## 9. Scaling & Encoding


In [44]:

scaler = StandardScaler()

df[['rate', 'approx_cost(for_two_people)', 'cuisine_count']] = scaler.fit_transform(
    df[['rate', 'approx_cost(for_two_people)', 'cuisine_count']]
)

# One-hot encoding
df = pd.get_dummies(df, columns=['location', 'rest_type'], drop_first=True)

dish_dummies = df['dish_liked_grouped'].str.get_dummies(sep=',')
df = pd.concat([df, dish_dummies], axis=1)

# Convert boolean columns
bool_cols = df.select_dtypes('bool').columns
df[bool_cols] = df[bool_cols].astype(int)


c:\Users\Phoollazy\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\extmath.py:1144: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
c:\Users\Phoollazy\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\extmath.py:1149: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
c:\Users\Phoollazy\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\extmath.py:1169: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count



## 10. Final Dataset Cleanup


In [45]:

df.drop(
    ['url', 'address', 'phone', 'reviews_list', 'menu_item'],
    axis=1,
    inplace=True
)

print("Final dataset shape:", df.shape)
df.head()


Final dataset shape: (51717, 39)


,name,online_order,book_table,rate,votes,dish_liked,cuisines,approx_cost(for_two_people),listed_in(type),listed_in(city),...,Burgers,Cocktails,Coffee,Mocktails,Nachos,Pasta,Pizza,Sandwiches,Burgers,Other
0,Jalsa,Yes,Yes,NaN,484.5,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",0.945764,Buffet,Banashankari,...,0,0,0,0,0,0,0,0,0,1
1,Spice Elephant,Yes,No,NaN,484.5,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",0.945764,Buffet,Banashankari,...,0,0,0,0,0,0,0,0,0,1
2,San Churro Cafe,Yes,No,NaN,484.5,"Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",0.945764,Buffet,Banashankari,...,0,0,0,0,0,0,0,0,0,1
3,Addhuri Udupi Bhojana,No,No,NaN,88.0,Masala Dosa,"South Indian, North Indian",-0.698524,Buffet,Banashankari,...,0,0,0,0,0,0,0,0,0,1
4,Grand Village,No,No,NaN,166.0,"Panipuri, Gol Gappe","North Indian, Rajasthani",0.288049,Buffet,Banashankari,...,0,0,0,0,0,0,0,0,0,1



## 11. Key Takeaways

- Real-world datasets require extensive cleaning
- Feature engineering adds more value than raw modeling
- Handling high-cardinality categories is critical
- The final dataset is fully model-ready
